## 1 原理说明
plan函数，主要是用来生成一个远程函数指针，使用本地操作，实现远程的操作。

就像进行梯度下降一样。我可能无法控制客户端执行的具体的代码，因为我只能传输数据。

这个时候可以传递plan计划，让客户端执行plan中的代码完成梯度下降和训练过程。

客户端可能不知道要做些什么行为，如何来训练模型，训练过程中的超参数，主要使用plan的方法，传递给客户端，让客户端执行操作。

1. 计划（Plan）指的是可存储的Torch操作序列，它可以被发送到远程机器执行，并且保留对其引用。它提出的目的是**减少通信量**。举之前例子，如果我们要反复在远程机器上完成两个张量的求和平均两个操作.每次计算都需要与远程机器通信一次，是不必要的开销。因此我们可以用计划包裹一系列操作，发送给工作机，然后只需要发一次消息即可。

2. 要将普通的函数转化为计划函数，只需要用**装饰器**即可实现.

3. 创建计划函数后，需要保证计划已经**被构建**才能使用，通过calcu.is_bulit进行判断。


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# 定义当前进程的worker为服务器进程
import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch ie add extra functionalities 


# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False


server = hook.local_worker

In [3]:
# 定义客户端worker
x11 = torch.tensor([-1, 2.]).tag('input_data')
x12 = torch.tensor([1, -2.]).tag('input_data2')
x21 = torch.tensor([-1, 2.]).tag('input_data')
x22 = torch.tensor([1, -2.]).tag('input_data2')

device_1 = sy.VirtualWorker(hook, id="device_1", data=(x11, x12)) 
device_2 = sy.VirtualWorker(hook, id="device_2", data=(x21, x22))
devices = device_1, device_2

In [4]:
# 定义计划函数
@sy.func2plan()
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

In [5]:
plan_double_abs

<Plan plan_double_abs id:95989162597 owner:me>
def plan_double_abs():
    return 

In [6]:
pointer_to_data = device_1.search('input_data')[0]
pointer_to_data

tensor([-1.,  2.])
	Tags: input_data 
	Shape: torch.Size([2])

In [7]:
plan_double_abs.is_built

False

In [8]:
# Sending non-built Plan will fail
try:
    plan_double_abs.send(device_1)
except RuntimeError as error:
    print(error)

A plan needs to be built before being sent to a worker.


In [9]:
# 使用设备上的数据构建plan计划
plan_double_abs.build(torch.tensor([1., -2.]))
plan_double_abs.is_built

True

In [10]:
# This cell is executed successfully
pointer_plan = plan_double_abs.send(device_1)
pointer_plan

[PointerPlan | me:99455482860 -> device_1:95989162597]

In [11]:
# 运行指针上的计算
pointer_to_result = pointer_plan(pointer_to_data)
print(pointer_to_result)

(Wrapper)>[PointerTensor | me:71552086868 -> device_1:41296790067]


In [12]:
pointer_to_result.get()

tensor([2., 4.])

## 2 具体实例

减少通信的方法：通过计划，

In [13]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [14]:
net = Net()
print(net)

<Net Net id:55855385282 owner:me>
def Net():
    return 


In [15]:
# 使用数据构建计划
net.build(torch.tensor([1., 2.]))

In [16]:
# 将数据发送给远程设备
pointer_to_net = net.send(device_1)
pointer_to_net


[PointerPlan | me:96392609349 -> device_1:55855385282]

In [17]:
pointer_to_data = device_1.search('input_data')[0]

In [18]:
pointer_to_result = pointer_to_net(pointer_to_data)
pointer_to_result

(Wrapper)>[PointerTensor | me:25850645014 -> device_1:63083167324]

In [19]:
pointer_to_result.get()

tensor([-1.3050, -0.3163], requires_grad=True)

## 3 在工作人员之间进行切换

In [20]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [21]:
net = Net()

# Build plan
net.build(torch.tensor([1., 2.]))

In [23]:
# 在设备1上构建计划
pointer_to_net_1 = net.send(device_1)
pointer_to_data = device_1.search('input_data')[0]
pointer_to_result = pointer_to_net_1(pointer_to_data)
pointer_to_result.get()

tensor([-0.6172, -0.7753], requires_grad=True)

In [24]:
# 在设备2上构建计划
pointer_to_net_2 = net.send(device_2)
pointer_to_data = device_2.search('input_data')[0]
pointer_to_result = pointer_to_net_2(pointer_to_data)
pointer_to_result.get()

tensor([-0.6172, -0.7753], requires_grad=True)

In [25]:
## 4 自动构建计划


In [26]:
@sy.func2plan(args_shape=[(-1, 1)])
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

plan_double_abs.is_built

True

In [27]:
@sy.func2plan(args_shape=[(1, 2), (-1, 2)])
def plan_sum_abs(x, y):
    s = x + y
    return torch.abs(s)

plan_sum_abs.is_built

True

In [28]:
@sy.func2plan(args_shape=[(1,)], state=(torch.tensor([1]), ))
def plan_abs(x, state):
    bias, = state.read()
    x = x.abs()
    return x + bias

In [29]:
pointer_plan = plan_abs.send(device_1)
x_ptr = torch.tensor([-1, 0]).send(device_1)
p = pointer_plan(x_ptr)
p.get()

tensor([2, 1])